In [1]:
%load_ext autoreload
%autoreload 2


In [2]:

import os

from dotenv import load_dotenv
load_dotenv()

from hud import gym, load_taskset
from hud.agent import ClaudeAgent

from anthropic import Anthropic

In [7]:
taskset = await load_taskset("OSWorld-Ubuntu-Links")
print(f"Total tasks in OSWorld: {len(taskset)}")

Total tasks in OSWorld: 369


In [ ]:
# This will take around 2 minutes to start, but can be parallelized
env = await gym.make(taskset[1])

In [ ]:
# Reset to a task with an observation (screenshot and text)
obs, info = await env.reset()
print(f"Task description: {obs.text}")

In [ ]:
# Agent
anthropic = Anthropic(api_key=os.getenv("ANTHROPIC_API_KEY"))
agent = ClaudeAgent(anthropic)

# Agent loop
for i in range(8):
    print(f"========= Step {i+1} =========")
    action, done = await agent.predict(obs)
    print(f"Agent's action: {action}")

    if done:
        print("done")
        break

    obs, reward, terminated, info = await env.step(action)

    if terminated:
        print("terminated")
        break


In [ ]:
# Evaluate environment state
result = await env.evaluate()
print(f"Evaluation result: {result[0]['logs']}")

In [ ]:
# Make sure to close environment to avoid being charged for idle time
await env.close()